In [1]:
# imports
import pandas as pd
import const

# Preprocess 'all.xlsx' and 'mayors.xlsx'

In [2]:
# load data from xlsx
original_all_df = pd.read_excel('data/all.xlsx')
original_mayors_df = pd.read_excel('data/mayors.xlsx')

In [3]:
all_df = original_all_df.copy()
mayors_df = original_mayors_df.copy()

# load the new column names from the excel file
cols_df = pd.read_excel('data/new_cols_names.xlsx')

# Create a dictionary to map Hebrew column names to the new English names
col_mapping = dict(zip(cols_df['col_name_hebrew'], cols_df['new_col_name']))

# Rename the columns in all_df using the mapping
all_df.rename(columns=col_mapping, inplace=True)

## deal with "null" vals

In [4]:
all_df.district = all_df.district.str.replace('-', 'אזור יהודה והשומרון').fillna('אזור יהודה והשומרון')

all_df = all_df.replace('-', None).replace('..', None).replace('ללא דירוג', None)

for col in const.numeric_cols:
    all_df[col] = all_df[col].astype(float)

## map hebrew categories to categorial vals

In [5]:
disrict_names_mapping = {n: i for i, n in enumerate(all_df.district.unique())}
all_df.district = all_df.district.map(disrict_names_mapping)

In [6]:
# map values

mayors_df.location = mayors_df.location.replace(
    {
        'תל אביב': 'תל אביב -יפו',
        'נהריה': 'נהרייה',
    }
)
mayors_df.incident_type = mayors_df.incident_type.replace(
    {
        'הרשעה': 'conviction', 
        'הגשת כתב אישום': 'indictment filed', 
        'מעצר': 'arrest'
    }
)

## filter arab authorities

In [7]:
arabs_authority_code_l = all_df[all_df['arabs'] == 100].drop_duplicates(subset=['authority_code'])['authority_code'].tolist()
all_df = all_df[~all_df['authority_code'].isin(arabs_authority_code_l)]

# drop the 'arabs' column: we don't need it anymore
all_df.drop(columns=['arabs'], inplace=True)

## refine numeric columns

In [8]:
def percentage_cols_converter(df):
    for col_name in df.columns:
        if 'percent' in col_name.lower():
            # check if percentage and if not convert to None
            df[col_name] = df[col_name].apply(lambda x: x if isinstance(x, (int, float)) else -1)
            df[col_name] = df[col_name].apply(lambda x: x if 0.0 <= x <= 100.0 else None)
    return df

def numeric_cols_converter(df):
    cols_df = pd.read_excel('data/new_cols_names.xlsx')
    col_type_mapping = dict(zip(cols_df['new_col_name'], cols_df['col_type']))

    for col_name in df.columns:
        # Ensure the column is in the mapping before converting
        if col_name in col_type_mapping:
            if col_type_mapping[col_name] in ['int', 'float']:
                # Convert to numeric, coerce invalid values to NaN
                df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
            elif col_type_mapping[col_name] == 'category':
                # Handle district similarly with numeric coercion and Int64 casting
                df[col_name] = df[col_name].astype('int')
            else:
                print(f"Unknown column type in {col_name}: {col_type_mapping[col_name]}")
    
    return df

In [9]:
all_df = percentage_cols_converter(all_df)
all_df = numeric_cols_converter(all_df)

Unknown column type in authority_name: str


## Normalize the column values by checking if there's a large jump in values after a specific year.

In [10]:
def fix_values_scale(df, col_name, year_col='year', threshold_year=2018):
    """
    Fix the scale of values in a column based on the median value before and after a threshold year.
    
    (the problem is that the values after the threshold year are approximately 1000 times larger than before)
    """
    # Check the median value before the threshold year
    median_before = df[df[year_col] < threshold_year][col_name].median()
    
    # Check the median value after the threshold year
    median_after = df[df[year_col] >= threshold_year][col_name].median()

    # If the values after the threshold year are approximately 1000 times larger
    if median_after > median_before * 100:
        print(col_name)
        # Scale the values after the threshold year by 1000
        df.loc[df[year_col] >= threshold_year, col_name] /= 1000.0
    
    return df


In [11]:
cols_df = pd.read_excel('data/new_cols_names.xlsx')
col_type_mapping = dict(zip(cols_df['new_col_name'], cols_df['col_type']))

for col_name in ['total_population', 'total_men', 'total_women']:    
    if col_name in col_type_mapping:
        if col_type_mapping[col_name] in ['int', 'float']:
            all_df = fix_values_scale(all_df, col_name)

total_population
total_men
total_women


## assign constant values that doesnt change in time (distance_from_tel_aviv, year_of_municipal_status)

In [12]:
# assign the distance from tel aviv to each authority code as the mean of the distance from tel aviv in available data
auth_code_to_dist_from_tlv = all_df.groupby('authority_code')['distance_from_tel_aviv'].mean()

# take the minimal in year_of_municipal_status col
auth_code_to_muni_status_yr = all_df[['authority_code','year_of_municipal_status']].drop_duplicates()\
    .groupby('authority_code')['year_of_municipal_status'].min()

In [13]:
# map the values of distance_from_tel_aviv and year_of_municipal_status so they will be the same for every city in all years.
all_df['distance_from_tel_aviv'] = all_df['authority_code'].map(auth_code_to_dist_from_tlv)
all_df['year_of_municipal_status'] = all_df['authority_code'].map(auth_code_to_muni_status_yr)

## remove authorities with missing years in data

In [14]:
# remove authorities with missing years in data
auth_to_num_years = all_df[['authority_code', 'year']].drop_duplicates().groupby('authority_code')['year'].agg(['count']).reset_index()
auth_missing_too_many_years = auth_to_num_years[auth_to_num_years['count'] < 23]['authority_code'].tolist()
all_df = all_df[~all_df['authority_code'].isin(auth_missing_too_many_years)]

In [15]:
df_all_years = pd.DataFrame({'year': range(all_df['year'].min(), all_df['year'].max() + 1)})
df_all_codes = pd.DataFrame({'authority_code': all_df['authority_code'].unique()})
df_all_years_and_codes = df_all_years.assign(key=1).merge(df_all_codes.assign(key=1), on='key').drop(columns='key')

# merge all years and all codes to the data
all_df = df_all_years_and_codes.merge(all_df, on=['year', 'authority_code'], how='left')


# fill missing values


In [16]:
def fill_na_with_col_mean(df, col_name):
    df_copy = df.copy()
    # Group by 'authority_code' and calculate the mean of the 'col_name' column for each group (ignoring NaNs)
    df_copy[col_name] = df_copy.groupby('authority_code')[col_name].transform(lambda x: x.fillna(x.mean()))
    return df_copy


def fill_na_with_prev_years_mean(df, col_name, num_years=3):
    df_copy = df.copy()

    # Sort by authority_code and year to ensure rolling works properly
    df_copy = df_copy.sort_values(by=['authority_code', 'year'])

    # Define a function that fills NaNs for a specific column using rolling mean of previous 3 years
    def fill_with_rolling_mean(group):
        filler = group[col_name].rolling(window=num_years, min_periods=1).mean()
        group[col_name] = group[col_name].fillna(filler)
        return group

    # Group by 'authority_code' and apply the function to the 'value' column
    df_copy = df_copy.groupby('authority_code').apply(fill_with_rolling_mean)

    return df_copy

def fill_na_by_backfill(df, col_name):
    df_copy = df.copy()
    df_copy[col_name] = df_copy.groupby('authority_code')[col_name].bfill()
    return df_copy

def fill_na_by_fillforward(df, col_name):
    df_copy = df.copy()
    df_copy[col_name] = df_copy.groupby('authority_code')[col_name].ffill()
    return df_copy

def fill_na_vals_with_means(df, cols_to_fillna):
    df_copy = df.copy()
    for col_name in cols_to_fillna:
        # first fill na with the mean of prev 3 years
        df_copy = fill_na_with_prev_years_mean(df_copy, col_name)
        # second fill na with the mean of the column
        df_copy = fill_na_with_col_mean(df_copy, col_name)
    return df_copy

def fill_na_vals_with_forward_than_backfill(df, cols_to_fillna):
    df_copy = df.copy()
    for col_name in cols_to_fillna:
        # first fillfaorward
        df_copy = fill_na_by_fillforward(df_copy, col_name)
        # second backfill
        df_copy = fill_na_by_backfill(df_copy, col_name)
    return df_copy

def fill_na_vals_with_backfill_than_forward(df, cols_to_fillna):
    df_copy = df.copy()
    for col_name in cols_to_fillna:
        # first backfill
        df_copy = fill_na_by_backfill(df_copy, col_name)
        # second fillforward
        df_copy = fill_na_by_fillforward(df_copy, col_name)
    return df_copy

def fill_na_vals_with_means_than_forward_than_back(df, cols_to_fillna):
    df_copy = df.copy()

    # first fillna by 3 years mean
    for col_name in cols_to_fillna:
        # first fill na with the mean of prev 3 years
        df_copy = fill_na_with_prev_years_mean(df_copy, col_name)
        # first fillfaorward
        df_copy = fill_na_by_fillforward(df_copy, col_name)
        # second backfill
        df_copy = fill_na_by_backfill(df_copy, col_name)
    # # second fillna with forward than backfill
    # df_copy = fill_na_vals_with_forward_than_backfill(df_copy, cols_to_fillna=cols_to_fillna)
    return df_copy


#### fill na values: matching cols

In [17]:
cols_to_filna_by_3years_mean_than_col_mean = [
    'council_members_number',
    'total_migration_balance', 
    'immigrants_post_1990', 
    'population_growth_rate', 
    'total_students', 
    'avg_students_per_class', 
    'percent_bagrut_eligible', 
    'percent_university_bagrut', 
    'water_receipts', 
    'total_private_cars', 
    'avg_car_age', 
    'accidents_per_1000_residents', 
    'accidents_per_1000_vehicles', 
    'unemployment_benefits_total', 
    'male_unemployment_benefits', 
    'avg_age_unemployment_recipients', 
    'income_support_recipients_yearly', 
    'avg_salary_employees', 
    'avg_salary_male', 
    'avg_salary_female', 
    'change_salary_emp_yoy', 
    'change_salary_male_yoy', 
    'change_salary_female_yoy', 
    'employees_total', 
    'percent_earning_min_wage', 
    'self_employed_total', 
    'avg_income_self_employed', 
    'change_income_self_employed_yoy', 
    'percent_earning_half_avg_salary', 
    'inequality_index_gini', 
    'total_revenues', 
    'revenues_change', 
    'regular_budget_revenues', 
    'gov_participation', 
    'gov_participation_change', 
    'special_budget_revenues', 
    'property_tax_total_prev_year', 
    'property_tax_residential_prev_year', 
    'property_tax_business_prev_year', 
    'property_tax_total_current_year', 
    'property_tax_residential_current_year', 
    'property_tax_business_current_year', 
]

cols_to_forwardfill_then_back = [
    'socioeconomic_level',
    'compactness_cluster',
    'peripheral_cluster', 
    'total_jurisdiction_area',
    'population_density',
    'total_population', # no nans
    'jews_and_others', # no nans
    'total_men', # no nans
    'total_women', # no nans
    'dependency_ratio',
    'natural_increase_per_1000',
    'residential_percent_area', 
    'education_percent_area', 
    'health_welfare_percent_area', 
    'public_services_percent_area', 
    'culture_leisure_sports_percent_area', 
    'commerce_percent_area', 
    'industry_infrastructure_percent_area', 
    'transportation_percent_area', 
    'agricultural_buildings_percent_area', 
    'public_open_space_percent_area', 
    'forest_woodland_percent_area', 
    'orchards_citrus_olive_percent_area', 
    'cultivated_fields_percent_area', 
    'other_open_space_percent_area', 
    'total_classes', 
]
cols_to_filna_backfill = [
    'immigrants_post_1990'
]

all_matching_cols = [
    'district', 
    'distance_from_tel_aviv', 
    'year_of_municipal_status', 
    'council_members_number', 
    'socioeconomic_level', 
    'compactness_cluster', 
    'peripheral_cluster', 
    'total_jurisdiction_area', 
    'population_density', 
    'total_population', 
    'jews_and_others', 
    'arabs', 
    'total_men', 
    'total_women', 
    'dependency_ratio', 
    'natural_increase_per_1000', 
    'total_migration_balance', 
    'immigrants_post_1990', 
    'population_growth_rate', 
    'residential_percent_area', 
    'education_percent_area', 
    'health_welfare_percent_area', 
    'public_services_percent_area', 
    'culture_leisure_sports_percent_area', 
    'commerce_percent_area', 
    'industry_infrastructure_percent_area', 
    'transportation_percent_area', 
    'agricultural_buildings_percent_area', 
    'public_open_space_percent_area', 
    'forest_woodland_percent_area', 
    'orchards_citrus_olive_percent_area', 
    'cultivated_fields_percent_area', 
    'other_open_space_percent_area', 
    'total_schools', 
    'total_classes', 
    'total_students', 
    'avg_students_per_class', 
    'percent_bagrut_eligible', 
    'percent_university_bagrut', 
    'water_receipts', 
    'total_private_cars', 
    'avg_car_age', 
    'accidents_per_1000_residents', 
    'accidents_per_1000_vehicles', 
    'unemployment_benefits_total', 
    'male_unemployment_benefits', 
    'avg_age_unemployment_recipients', 
    'income_support_recipients_yearly', 
    'avg_salary_employees', 
    'avg_salary_male', 
    'avg_salary_female', 
    'change_salary_emp_yoy', 
    'change_salary_male_yoy', 
    'change_salary_female_yoy', 
    'employees_total', 
    'percent_earning_min_wage', 
    'self_employed_total', 
    'avg_income_self_employed', 
    'change_income_self_employed_yoy', 
    'percent_earning_half_avg_salary', 
    'inequality_index_gini', 
    'total_revenues', 
    'revenues_change', 
    'regular_budget_revenues', 
    'gov_participation', 
    'gov_participation_change', 
    'special_budget_revenues', 
    'property_tax_total_prev_year', 
    'property_tax_residential_prev_year', 
    'property_tax_business_prev_year', 
    'property_tax_total_current_year', 
    'property_tax_residential_current_year', 
    'property_tax_business_current_year', 
]

all_filled_na_cols = cols_to_filna_by_3years_mean_than_col_mean + cols_to_forwardfill_then_back + cols_to_filna_backfill

set(all_matching_cols) - set(all_filled_na_cols)

{'arabs',
 'distance_from_tel_aviv',
 'district',
 'total_schools',
 'year_of_municipal_status'}

In [18]:

all_df = fill_na_vals_with_means(all_df, cols_to_fillna=cols_to_filna_by_3years_mean_than_col_mean)
all_df = fill_na_vals_with_forward_than_backfill(all_df, cols_to_fillna=cols_to_forwardfill_then_back)
all_df = fill_na_vals_with_backfill_than_forward(all_df, cols_to_fillna=cols_to_filna_backfill)

/tmp/ipykernel_47205/2305848137.py:21: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_copy = df_copy.groupby('authority_code').apply(fill_with_rolling_mean)
/tmp/ipykernel_47205/2305848137.py:21: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_copy = df_copy.groupby('authority_code').apply(fi

In [19]:
# check filna worked: get lines with still nan values
all_df[all_filled_na_cols][all_df[all_filled_na_cols].isna().any(axis=1)][all_filled_na_cols]

# get cols with still nan values
# all_df.columns[all_df.isna().any()].tolist()

,council_members_number,total_migration_balance,immigrants_post_1990,immigrants_post_1990,population_growth_rate,total_students,avg_students_per_class,percent_bagrut_eligible,percent_university_bagrut,water_receipts,...,transportation_percent_area,agricultural_buildings_percent_area,public_open_space_percent_area,forest_woodland_percent_area,orchards_citrus_olive_percent_area,cultivated_fields_percent_area,other_open_space_percent_area,total_classes,immigrants_post_1990,immigrants_post_1990


#### fill na values: predict cols

In [20]:
##

### save df all cleaned

In [21]:
all_df.to_csv('data/all_cleaned.csv', index=False)

# join mayors


In [22]:
# muni_names_to_code_mapping = pd.merge(
#         mayors_df[['location']].drop_duplicates().reset_index(drop=True),
#         all_df[['authority_name', 'authority_code']].drop_duplicates(),
#         left_on='location', right_on='authority_name',
#         how='left'
#     ).dropna().reset_index(drop=True).drop(columns='authority_name')

# muni_names_to_code_mapping.authority_code = muni_names_to_code_mapping.authority_code.astype(int)
# muni_names_to_code_mapping = all_df[['authority_name', 'authority_code']].drop_duplicates().reset_index(drop=True)
# muni_names_to_code_mapping

In [23]:
# save mapping 
# muni_names_to_code_mapping.to_csv('data/muni_names_to_code_mapping.csv', index=False)

# load municipalities names to code mapping
muni_names_to_code_mapping = pd.read_csv('data/muni_names_to_code_mapping.csv')

# merge all data with the mapping on auth code, to fix names
all_df.drop(columns='authority_name', inplace=True)
all_df = all_df.merge(muni_names_to_code_mapping, on='authority_code', how='left')


In [24]:
muni_names_to_code_mapping.groupby('authority_code').size().max()

1

In [25]:
# preprocess mayors df columns
mayors_df['incident_year'] = mayors_df['date'].apply(lambda x: str(x).split('-')[0])
mayors_df = mayors_df[['incident_year', 'incident_type', 'location']]
mayors_df = mayors_df[mayors_df.incident_type.isin(['conviction', 'indictment filed', 'arrest'])]
mayors_df.drop_duplicates(inplace=True)
mayors_df.reset_index(drop=True, inplace=True)

In [26]:
# add the authority code to the mayors df
mayors_df = pd.merge(
    mayors_df,
    muni_names_to_code_mapping,
    left_on='location', right_on='authority_name',
    how='left'
).drop(columns='location').dropna(subset=['authority_code']).reset_index(drop=True)

In [27]:
mayors_df.authority_code = mayors_df.authority_code.astype(int)
mayors_df.drop(columns='authority_name', inplace=True)

### save the cleaned mayors df

In [28]:
#  save the cleaned mayors df
mayors_df.to_csv('data/mayors_cleaned.csv', index=False)

## merge the all data with the mayors data

In [29]:
# merge the data with the mayors data
all_df = pd.merge(
    all_df,
    mayors_df,
    on=['authority_code'],
    how='left'
)
all_df['is_treatment'] = all_df['incident_year'].notnull().astype(int)
# all_df.query('is_treatment == 1')


### save the final data

In [30]:
# # save the final data
all_df.to_csv('data/all_cleaned_with_mayors_data.csv', index=False)

# Prepare df for matching

In [31]:
# load needed data
all_df = pd.read_csv('data/all_cleaned_with_mayors_data.csv')

# load the column details from the excel file
original_cols_df = pd.read_excel('data/new_cols_names.xlsx')
cols_df = original_cols_df.copy()

In [32]:
# get matching cols only
cols_df.matching = cols_df.matching.fillna(0).astype(bool)

matching_cols = ['year', 'authority_code', 'authority_name', 'is_treatment'] + cols_df[cols_df.matching]['new_col_name'].tolist()
matching_cols.remove('total_schools')
matching_cols.remove('arabs')
len(matching_cols)

75

In [33]:
matching_df = all_df[matching_cols].drop_duplicates().reset_index(drop=True)

In [34]:
# save matching df
matching_df.to_csv('data/matching_features_df.csv', index=False)

# Prepare df for inference

In [35]:
# load needed data
all_df = pd.read_csv('data/all_cleaned_with_mayors_data.csv')
all_df.year = all_df.year.astype(int)
all_df.incident_year = all_df.incident_year.apply(lambda x: int(x) if pd.notna(x) else x)

# load the column details from the excel file
original_cols_df = pd.read_excel('data/new_cols_names.xlsx')
cols_df = original_cols_df.copy()
cols_df.predicting = cols_df.predicting.fillna(0).astype(bool)

# load the mayors df
mayors_df = pd.read_csv('data/mayors_cleaned.csv')

## fill na predicting cols

In [36]:
all_inference_cols = cols_df[cols_df.predicting]['new_col_name'].tolist()

# remove cols that have only one value in 2007:  
cols_to_remove = [
    'satisfaction_municipal_services', 
    'satisfaction_municipal_performance',
    'satisfaction_residential_area',
    'satisfaction_cleanliness',
    'satisfaction_parks_green_spaces',
    'feeling_safe_dark',
    'contacting_local_authority',
]
all_inference_cols = [col for col in all_inference_cols if col not in cols_to_remove]


In [37]:
inference_df_cols = ['year', 'is_treatment'] + mayors_df.columns.tolist() + all_inference_cols

inference_df = all_df[inference_df_cols].drop_duplicates().reset_index(drop=True)
inference_df


,year,is_treatment,incident_year,incident_type,authority_code,total_in_migration,male_in_migration,female_in_migration,in_migration_age_15_29,total_out_migration,...,complete_drainage_pipes_km,total_expenditures,expenditures_change,operations_expenditures,loan_repayment_expenditures,financing_expenditures,special_budget_expenditures,budget_surplus_deficit,annual_surplus_deficit,cumulative_deficit
0,1999,0,NaN,NaN,26,127.0,67.0,60.0,41.0,87.0,...,NaN,22876.0,29.5,8470.0,NaN,129.0,NaN,261.0,261.0,NaN
1,2000,0,NaN,NaN,26,79.0,37.0,42.0,21.0,81.0,...,NaN,20864.0,-9.8,9657.0,NaN,31.0,NaN,-75.0,-75.0,NaN
2,2001,0,NaN,NaN,26,92.0,46.0,46.0,23.0,99.0,...,NaN,19626.0,-7.0,4273.0,NaN,200.0,NaN,-275.0,-692.0,645.0
3,2002,0,NaN,NaN,26,129.0,52.0,77.0,30.0,82.0,...,NaN,22893.0,10.4,4839.0,1230.0,162.0,5069.0,78.0,-562.0,723.0
4,2003,0,NaN,NaN,26,130.0,61.0,69.0,32.0,82.0,...,NaN,25956.0,12.6,6129.0,1061.0,122.0,7154.0,363.0,23.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283,2018,0,NaN,NaN,9800,579.0,283.0,296.0,66.0,595.0,...,NaN,121310.0,NaN,38764.0,4279.0,738.0,23341.0,23.0,23.0,-14389.0
3284,2019,0,NaN,NaN,9800,640.0,335.0,305.0,83.0,638.0,...,NaN,112229.0,NaN,41666.0,4379.0,859.0,9143.0,-123.0,-123.0,-14512.0
3285,2020,0,NaN,NaN,9800,527.0,237.0,290.0,76.0,563.0,...,NaN,125897.0,NaN,40176.0,3619.0,818.0,20595.0,-1367.0,-1367.0,-15879.0
3286,2021,0,NaN,NaN,9800,548.0,279.0,269.0,63.0,543.0,...,NaN,143244.0,NaN,44171.0,4104.0,866.0,32026.0,24.0,24.0,-10501.0


In [38]:
# save inference df without imputation
inference_df.to_csv('data/inference_features_df.csv', index=False)


In [39]:
inference_df.columns

Index(['year', 'is_treatment', 'incident_year', 'incident_type',
       'authority_code', 'total_in_migration', 'male_in_migration',
       'female_in_migration', 'in_migration_age_15_29', 'total_out_migration',
       'male_out_migration', 'female_out_migration', 'out_migration_age_15_29',
       'start_roads_length_km', 'start_roads_area_sqm', 'start_water_pipes_km',
       'start_sewage_pipes_km', 'start_drainage_pipes_km',
       'complete_roads_length_km', 'complete_roads_area_sqm',
       'complete_water_pipes_km', 'complete_sewage_pipes_km',
       'complete_drainage_pipes_km', 'total_expenditures',
       'expenditures_change', 'operations_expenditures',
       'loan_repayment_expenditures', 'financing_expenditures',
       'special_budget_expenditures', 'budget_surplus_deficit',
       'annual_surplus_deficit', 'cumulative_deficit'],
      dtype='object')

In [40]:

# fill na predicting cols
cols_to_filna_by_3years_mean_than_col_mean = [
    'total_in_migration',
    'male_in_migration',
    'female_in_migration',
    'in_migration_age_15_29',
    'total_out_migration',
    'male_out_migration',
    'female_out_migration',
    'out_migration_age_15_29',
    'total_expenditures',
    'operations_expenditures',
    'loan_repayment_expenditures',
    'financing_expenditures',
    'special_budget_expenditures',
    'budget_surplus_deficit',
    'annual_surplus_deficit', 
]
cols_to_forwardfill_then_back = [
    'expenditures_change',
    'cumulative_deficit',

]
cols_to_filna_by_3years_mean_than_forwardfill_than_backfill = [
    'start_roads_length_km',
    'start_roads_area_sqm',
    'start_water_pipes_km',
    'start_sewage_pipes_km',
    'start_drainage_pipes_km',
    'complete_roads_length_km',
    'complete_roads_area_sqm',
    'complete_water_pipes_km',
    'complete_sewage_pipes_km',
    'complete_drainage_pipes_km',
]

# check
all_filled_na_cols = cols_to_filna_by_3years_mean_than_forwardfill_than_backfill + cols_to_forwardfill_then_back + cols_to_filna_by_3years_mean_than_col_mean
set(all_inference_cols) - set(all_filled_na_cols) , set(all_filled_na_cols) - set(all_inference_cols)

(set(), set())

In [41]:
all_df_filled = pd.DataFrame()
for t, cur_df in all_df.groupby('incident_type'):
    if t is None:
        continue

    # Separate dataframes for years <= incident_year and > incident_year
    df_before_incident = cur_df[cur_df['year'] <= cur_df['incident_year']]
    df_after_incident = cur_df[cur_df['year'] > cur_df['incident_year']]

    # Process df_before_incident
    df_filled_before = fill_na_vals_with_means_than_forward_than_back(df=df_before_incident, cols_to_fillna=cols_to_filna_by_3years_mean_than_forwardfill_than_backfill)
    df_filled_before = fill_na_vals_with_forward_than_backfill(df=df_filled_before, cols_to_fillna=cols_to_forwardfill_then_back)
    df_filled_before = fill_na_vals_with_means(df=df_filled_before, cols_to_fillna=cols_to_filna_by_3years_mean_than_col_mean)

    # Process df_after_incident
    df_filled_after = fill_na_vals_with_means_than_forward_than_back(df=df_after_incident, cols_to_fillna=cols_to_filna_by_3years_mean_than_forwardfill_than_backfill)
    df_filled_after = fill_na_vals_with_forward_than_backfill(df=df_filled_after, cols_to_fillna=cols_to_forwardfill_then_back)
    df_filled_after = fill_na_vals_with_means(df=df_filled_after, cols_to_fillna=cols_to_filna_by_3years_mean_than_col_mean)

    # Combine the processed dataframes
    df_filled = pd.concat([df_filled_before, df_filled_after])

    # Update the original dataframe
    all_df_filled = pd.concat([all_df_filled, df_filled])

# fillna where incident_year is nan 
df_control_auth = all_df[all_df.incident_year.isna()]
df_control_auth = fill_na_vals_with_means_than_forward_than_back(df=df_control_auth, cols_to_fillna=cols_to_filna_by_3years_mean_than_forwardfill_than_backfill)
df_control_auth = fill_na_vals_with_forward_than_backfill(df=df_control_auth, cols_to_fillna=cols_to_forwardfill_then_back)
df_control_auth = fill_na_vals_with_means(df=df_control_auth, cols_to_fillna=cols_to_filna_by_3years_mean_than_col_mean)

# add to all_df
all_df_filled = pd.concat([all_df_filled, df_control_auth])


/tmp/ipykernel_47205/2305848137.py:21: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_copy = df_copy.groupby('authority_code').apply(fill_with_rolling_mean)
/tmp/ipykernel_47205/2305848137.py:21: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_copy = df_copy.groupby('authority_code').apply(fi

In [42]:
# check filna worked: get lines with still nan values
# df_nas = all_df_filled[all_inference_cols][all_df_filled[all_inference_cols].isna().any(axis=1)][all_inference_cols]
all_df_filled[all_inference_cols].isna().any(axis=1).sum()

# NOTE: the remaining nas are if there is no data in the before\after incident years...


840

#### save inference df


In [43]:
inference_df_cols = ['year', 'is_treatment'] + mayors_df.columns.tolist() + all_inference_cols

inference_df_imputed = all_df_filled[inference_df_cols].drop_duplicates().reset_index(drop=True)
inference_df_imputed

,year,is_treatment,incident_year,incident_type,authority_code,total_in_migration,male_in_migration,female_in_migration,in_migration_age_15_29,total_out_migration,...,complete_drainage_pipes_km,total_expenditures,expenditures_change,operations_expenditures,loan_repayment_expenditures,financing_expenditures,special_budget_expenditures,budget_surplus_deficit,annual_surplus_deficit,cumulative_deficit
0,1999,1,2012.0,arrest,1034,546.0,267.0,279.0,228.0,953.0,...,1.8,131211.0,-8.389252,45087.0,12581.0,2381.0,18593.0,-6822.55,-8376.95,-56543.0
1,2000,1,2012.0,arrest,1034,454.0,231.0,223.0,222.0,837.0,...,1.8,129228.0,-2.621422,43577.0,13265.0,1777.0,22722.0,-6822.55,-8376.95,-56543.0
2,2001,1,2012.0,arrest,1034,530.0,265.0,265.0,217.0,1057.0,...,1.8,127896.0,-2.107553,17803.0,12923.0,2010.0,16270.0,-6822.55,-8376.95,-56543.0
3,2002,1,2012.0,arrest,1034,492.0,248.0,244.0,219.5,947.0,...,1.8,128562.0,-2.107553,30690.0,13265.0,1893.5,19496.0,-6822.55,-8376.95,-56543.0
4,2003,1,2012.0,arrest,1034,588.0,301.0,287.0,185.0,709.0,...,1.8,135914.0,6.921554,40738.0,14982.0,4102.0,16560.0,-19856.00,-19856.00,-56543.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283,2018,0,NaN,NaN,9800,579.0,283.0,296.0,66.0,595.0,...,0.3,121310.0,2.100000,38764.0,4279.0,738.0,23341.0,23.00,23.00,-14389.0
3284,2019,0,NaN,NaN,9800,640.0,335.0,305.0,83.0,638.0,...,0.3,112229.0,2.100000,41666.0,4379.0,859.0,9143.0,-123.00,-123.00,-14512.0
3285,2020,0,NaN,NaN,9800,527.0,237.0,290.0,76.0,563.0,...,0.3,125897.0,2.100000,40176.0,3619.0,818.0,20595.0,-1367.00,-1367.00,-15879.0
3286,2021,0,NaN,NaN,9800,548.0,279.0,269.0,63.0,543.0,...,0.3,143244.0,2.100000,44171.0,4104.0,866.0,32026.0,24.00,24.00,-10501.0


In [44]:
# save inference df
inference_df_imputed.to_csv('data/inference_features_df_imputed.csv', index=False)